<a href="https://colab.research.google.com/github/zhuodicai/music-study/blob/main/midi%2Bsinging_synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create midi + play

## Install mingus + MIDIUtil

In [ ]:
!pip install mingus
!pip install MIDIUtil

## Install musescore

In [ ]:
!yes | add-apt-repository ppa:mscore-ubuntu/mscore3-stable
!apt update
!apt install musescore3
!ln -s /usr/bin/musescore3 /usr/bin/musescore

 ~~~~~~~~~ MUSESCORE 3 RELEASES FOR UBUNTU 18.04 AND LATER ~~~~~~~~~
INSTALLING: (run these commands from the terminal)
  sudo add-apt-repository ppa:mscore-ubuntu/mscore3-stable
  sudo apt-get update
  sudo apt-get install musescore3

DO *NOT* USE THESE PPAs on Debian, only on Ubuntu!

‣‣‣ These packages are NOT suitable for ”KDE neon 18.04 (bionic)”!
‣ Use https://build.opensuse.org/project/show/home:mirabile:mscore instead.
↓ Newer versions, such as “KDE neon 20.04 (focal)”, should work. If not, write me.

See https://launchpad.net/~mscore-ubuntu/+archive/ubuntu/mscore-stable
if you need to install MuseScore 2 (the older version) for some reason.

Note: MuseScore 3 cannot be made available for releases older than
Ubuntu 18.04 (“bionic”) LTS.

                                GETTING HELP:
 * Have a look at the Online Handbook: https://musescore.org/handbook
 * Try Google. Search for "musescore" and the problem you are having.
 * If those options fail you can try asking on the forum (

## Create midi

In [ ]:
from midiutil import MIDIFile
from mingus.core import chords

chord_progression = ["Cmaj7", "Cmaj7", "Fmaj7", "Gdom7"]

NOTES = ['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B']
OCTAVES = list(range(11))
NOTES_IN_OCTAVE = len(NOTES)

errors = {
    'notes': 'Bad input, please refer this spec-\n'
}


def swap_accidentals(note):
    if note == 'Db':
        return 'C#'
    if note == 'D#':
        return 'Eb'
    if note == 'E#':
        return 'F'
    if note == 'Gb':
        return 'F#'
    if note == 'G#':
        return 'Ab'
    if note == 'A#':
        return 'Bb'
    if note == 'B#':
        return 'C'

    return note


def note_to_number(note: str, octave: int) -> int:
    note = swap_accidentals(note)
    assert note in NOTES, errors['notes']
    assert octave in OCTAVES, errors['notes']

    note = NOTES.index(note)
    note += (NOTES_IN_OCTAVE * octave)

    assert 0 <= note <= 127, errors['notes']

    return note


array_of_notes = []
for chord in chord_progression:
    array_of_notes.extend(chords.from_shorthand(chord))

array_of_note_numbers = []
for note in array_of_notes:
    OCTAVE = 4
    array_of_note_numbers.append(note_to_number(note, OCTAVE))

track = 0
channel = 0
time = 0  # In beats
duration = 1  # In beats
tempo = 120  # In BPM
volume = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
# automatically)
MyMIDI.addTempo(track, time, tempo)

for i, pitch in enumerate(array_of_note_numbers):
    MyMIDI.addNote(track, channel, pitch, time + i, duration, volume)

with open("output.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

print("chord_progression:", chord_progression)
print("chord_progression:", chords.from_shorthand(chord_progression))

print("array_of_notes:", array_of_notes)
print("array_of_note_numbers:", array_of_note_numbers)

chord_progression: ['Cmaj7', 'Cmaj7', 'Fmaj7', 'Gdom7']
chord_progression: [['C', 'E', 'G', 'B'], ['C', 'E', 'G', 'B'], ['F', 'A', 'C', 'E'], ['G', 'B', 'D', 'F']]
array_of_notes: ['C', 'E', 'G', 'B', 'C', 'E', 'G', 'B', 'F', 'A', 'C', 'E', 'G', 'B', 'D', 'F']
array_of_note_numbers: [48, 52, 55, 59, 48, 52, 55, 59, 53, 57, 48, 52, 55, 59, 50, 53]


## Play midi

In [ ]:
!pip uninstall music21 --yes
!pip install music21
from music21 import note

Found existing installation: music21 9.1.0
Uninstalling music21-9.1.0:
  Successfully uninstalled music21-9.1.0
  Using cached music21-9.1.0-py3-none-any.whl (22.8 MB)


In [ ]:
from music21 import midi
mf = midi.MidiFile()
mf.open('output.mid') # path='abc.midi'
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('midi')

# Audio to midi

In [ ]:
!pip install sound_to_midi



In [ ]:
import sys
import librosa

from sound_to_midi.monophonic import wave_to_midi

print("Starting...")
# file_in = sys.argv[1]
# file_out = sys.argv[2]
file_in = "./voicetomidi_input1.wav"
file_out = "./voicetomidi_output1.mid"
y, sr = librosa.load(file_in, sr=None)
print(sr)


Starting...
48000


In [ ]:
print("Audio file loaded!")
midi = wave_to_midi(y)
print("Conversion finished!")
with open (file_out, 'wb') as f:
    midi.writeFile(f)
print("Done. Exiting!")

Audio file loaded!
Conversion finished!
Done. Exiting!


# Synthesize + play

## Install midi2voice
This is a [small library](https://pypi.org/project/midi2voice/) that makes it easy to interact with the [sinsy.jp](sinsy.jp) website. It implements a lyrics tokenization system so you can just provide a text file with the lyrics, then midi2voice maps it to the notes in the song for you.

In [ ]:
!pip install git+https://github.com/mathigatti/midi2voice.git

  Cloning https://github.com/mathigatti/midi2voice.git to /tmp/pip-req-build-qxoghba4
  Running command git clone --filter=blob:none --quiet https://github.com/mathigatti/midi2voice.git /tmp/pip-req-build-qxoghba4
  Resolved https://github.com/mathigatti/midi2voice.git to commit ebd5650dbbaff3b25058e14cbdcca0c08167e494
  Preparing metadata (setup.py) ... done


## Let's synthesize!
You can test it with the test files shallow.txt and shallow.mid or they can be replaced with whatever you want.

- Each row of the TXT represents 2 bars (8 beats). The synthesizer speaks English.

- MIDI file shouldn't have multiple notes playing at the same time never

In [ ]:
# Download file samples to test it
!wget -O shallow.mid https://github.com/mathigatti/midi2voice/blob/master/inputs/shallow.mid?raw=true
!wget -O shallow.txt https://raw.githubusercontent.com/mathigatti/midi2voice/master/inputs/shallow.txt

--2023-07-19 03:32:59--  https://github.com/mathigatti/midi2voice/blob/master/inputs/shallow.mid?raw=true
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/mathigatti/midi2voice/raw/master/inputs/shallow.mid [following]
--2023-07-19 03:33:00--  https://github.com/mathigatti/midi2voice/raw/master/inputs/shallow.mid
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mathigatti/midi2voice/master/inputs/shallow.mid [following]
--2023-07-19 03:33:00--  https://raw.githubusercontent.com/mathigatti/midi2voice/master/inputs/shallow.mid
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.


In [ ]:
%%time
# It might take a little bit
# !python -m midi2voice -l shallow.txt -m shallow.mid -g female -t 96
!python -m midi2voice -l shallow.txt -m output.mid -g male -t 96

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
convert <output.mid>...
	to <temp.xml>
... success!
CPU times: user 53.8 ms, sys: 5.38 ms, total: 59.2 ms
Wall time: 6.63 s


In [ ]:
import IPython.display as ipd
ipd.Audio("voice.wav")